In [3]:

#Install Detectron2 from the source
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-bw6bmpoi
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-bw6bmpoi
  Resolved https://github.com/facebookresearch/detectron2.git to commit ebe8b45437f86395352ab13402ba45b75b4d1ddb
  Preparing metadata (setup.py) ... done


In [1]:
#Set up directory and mounting from GDrive
import os
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#import libraries
import glob
import json
import os
import cv2
import numpy as np
import torch
print(torch.__version__)
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode


2.4.0+cu121


In [4]:
#install Google Earth Engine API
!pip install earthengine-api

In [4]:
import ee
ee.Authenticate()
ee.Initialize(project = 'southern-bonsai-435419-q2')

In [5]:
#adapted from https://github.com/csaybar/EEwPython/blob/master/10_Export.ipynb

import ee
import time

# Initialize the Earth Engine API
ee.Initialize()
#12.7049566, 53.5039470
# Define the geographic location and buffer
#Mareb
#longitude = 45.230503
#latitude = 15.376969

#Socotra
# longitude = 53.5039470
# latitude = 12.7049566


#Wadi Wajar Abyan
longitude = 46.156746
latitude = 13.941848

point = ee.Geometry.Point([longitude, latitude])
buffered_point = point.buffer(2000).bounds()
region = buffered_point.bounds()



# Date range
start_date = '1984-01-01'
end_date = '2022-12-31'

# Load and merge the updated image collections using Collection 2 data
landsat5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA').filterDate('1984-01-01', '2012-05-05')
landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA').filterDate('1999-01-01', '2022-12-31')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2013-01-01', '2022-12-31')
full_collection = landsat5.merge(landsat7).merge(landsat8)




# Filter out images with more than 50% cloud cover
cloud_filtered_collection = full_collection.filter(ee.Filter.lt('CLOUD_COVER', 50))

# Function to scale and cast images
def scale_and_cast(image):
    return image.multiply(255).byte()  # Scale and convert to uint8

# Apply the function to each image in the collection
filtered_collection = full_collection.filterBounds(region).select(['B4', 'B3', 'B2']).map(scale_and_cast)

# Create and start the video export task
task = ee.batch.Export.video.toDrive(**{
    'collection': filtered_collection,
    'description': 'Abyan2',
    'dimensions': 720,
    'framesPerSecond': 10,
    'region': region.coordinates().getInfo(),
    'folder': 'YourFolderName',
    'scale': 10,
    'maxFrames': 10000  # Increase the maxFrames as needed
})
task.start()

# Monitor the task
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)  # Sleep for 10 seconds before polling again

print("Video export complete.")


Polling for task (id: HXDE4SH2VWBKEN3OTOXR7J62).


KeyboardInterrupt: 

 ## Only using two periods of landsat

In [ ]:
#adapted from https://github.com/csaybar/EEwPython/blob/master/10_Export.ipynb

import ee
import time

# Initialize the Earth Engine API
ee.Initialize()
12.7049566, 53.5039470
# Define the geographic location and buffer
#Mareb
#longitude = 45.230503
#latitude = 15.376969

#Socotra
# longitude = 53.5039470
# latitude = 12.7049566


#Wadi Wajar Abyan
longitude = 46.156746
latitude = 13.941848

point = ee.Geometry.Point([longitude, latitude])
buffered_point = point.buffer(2000).bounds()
region = buffered_point.bounds()



# Date range
start_date = '1999-01-01'
end_date = '2022-12-31'

# Load and merge the updated image collections using Collection 2 data

landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA').filterDate('1999-01-01', '2022-12-31')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2013-01-01', '2022-12-31')
full_collection = landsat7.merge(landsat8)




# Filter out images with more than 50% cloud cover
cloud_filtered_collection = full_collection.filter(ee.Filter.lt('CLOUD_COVER', 50))

# Function to scale and cast images
def scale_and_cast(image):
    return image.multiply(255).byte()  # Scale and convert to uint8

# Apply the function to each image in the collection
filtered_collection = full_collection.filterBounds(region).select(['B4', 'B3', 'B2']).map(scale_and_cast)

# Create and start the video export task
task = ee.batch.Export.video.toDrive(**{
    'collection': filtered_collection,
    'description': 'Abyan2',
    'dimensions': 720,
    'framesPerSecond': 10,
    'region': region.coordinates().getInfo(),
    'folder': 'YourFolderName',
    'scale': 10,
    'maxFrames': 10000  # Increase the maxFrames as needed
})
task.start()

# Monitor the task
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)  # Sleep for 10 seconds before polling again

print("Video export complete.")


Polling for task (id: CLIH7VYRGMVOJ36BLPKZT7FE).
Polling for task (id: CLIH7VYRGMVOJ36BLPKZT7FE).
Video export complete.


## Loading the model

In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader, MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

# Register the dataset only if it's not already registered
#dataset_name = "waterbodies_test"


def get_waterbodies_dicts(img_dir, annotations_json):
    json_file = os.path.join(img_dir, annotations_json)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns['images']):
        record = {}
        filename = os.path.join(img_dir, v['file_name'])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = [anno for anno in imgs_anns['annotations'] if anno['image_id'] == v['id']]
        objs = []
        for anno in annos:
            obj = {
                "bbox": anno['bbox'],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": anno['segmentation'],
                "category_id": anno['category_id'],
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

# Paths to the train and test annotation files

# Register the datasets
# Paths to the train and test annotation files
annotation_file_test = '/content/drive/MyDrive/Project/annotations_test.json'
annotation_file_train = '/content/drive/MyDrive/Project/annotations_train.json'

# Register the datasets
DatasetCatalog.register("waterbodies_train", lambda: get_waterbodies_dicts("/content/drive/MyDrive/Project/images/", annotation_file_train))
MetadataCatalog.get("waterbodies_train").set(thing_classes=["waterbody"])

DatasetCatalog.register("waterbodies_test", lambda: get_waterbodies_dicts("/content/drive/MyDrive/Project/images/", annotation_file_test))
MetadataCatalog.get("waterbodies_test").set(thing_classes=["waterbody"])




# Initialize model
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Project/model_final.pth"

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.DATASETS.TEST = ("waterbodies_test",)
cfg.DATASETS.TRAIN = ("waterbodies_train",)  # Make sure no default COCO dataset is referenced
cfg.DATASETS.VAL = ("waterbodies_test",)  # Make sure no default COCO dataset is referenced
# cfg.INPUT.MIN_SIZE_TRAIN = (512,)  # Only one size, no choice needed
# cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
# cfg.INPUT.MAX_SIZE_TRAIN = 512  # No need to allow any larger sizes
# cfg.INPUT.MIN_SIZE_TEST = 512  # Same as training
# cfg.INPUT.MAX_SIZE_TEST = 512  # Same as training
# Setting up the test data loader and evaluator
val_loader = build_detection_test_loader(cfg, "waterbodies_test")
evaluator = COCOEvaluator("waterbodies_test", cfg, False, output_dir="/content/drive/MyDrive/Project/output/")


trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
# results = inference_on_dataset(trainer.model, val_loader, evaluator)
# print(results)
predictor = DefaultPredictor(cfg)

[09/14 21:25:12 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
| waterbody  | 569          |
|            |              |
[09/14 21:25:12 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/14 21:25:12 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/14 21:25:12 d2.data.common]: Serializing 569 elements to byte tensors and concatenating them all ...
[09/14 21:25:12 d2.data.common]: Serialized dataset takes 5.86 MiB
WARNING [09/14 21:25:12 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[09/14 21:25:12 d2.evaluation.coco_evaluation]: Trying to convert 'waterbodies_test' to COCO format ...
WARNING [09/14 21:25:13 d2.data.datasets.coco]: Using p

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[09/14 21:25:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[09/14 21:36:33 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/Project/model_final.pth ...


## Importing exported video from GDrive

In [5]:

#Taken from https://stackoverflow.com/questions/60663073/how-can-i-properly-run-detectron2-on-videos

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import libraries
import numpy as np
import tqdm
import cv2
# import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog
import time

# Extract video properties
video = cv2.VideoCapture('/content/drive/MyDrive/Timelapse videos/Input/lake-mead.mp4')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
video_writer = cv2.VideoWriter('/content/drive/MyDrive/Timelapse videos/Output/lake-meadout.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)



# Initialize visualizer
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

def runOnVideo(video, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """

    readFrames = 0
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break

        # Get prediction results for this frame
        outputs = predictor(frame)

        # Make sure the frame is colored
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Draw a visualization of the predictions using the video visualizer
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            break

# Create a cut-off for debugging
num_frames = 120

# Enumerate the frames of the video
for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):

    # Write test image
    cv2.imwrite('/content/drive/MyDrive/Timelapse videos/Output/TEST detectron2.png', visualization)

    # Write to video file
    video_writer.write(visualization)

# Release resources
video.release()
video_writer.release()
cv2.destroyAllWindows()

  0%|          | 0/120 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 32%|███▎      | 39/120 [01:37<03:23,  2.51s/it]
